In [1]:
import config as cfg
from tfpipe.core.utils import read_class_names
from tfpipe.pipeline.annotate_image import AnnotateImage
from tfpipe.pipeline.image_output import ImageOutput
import pickle
import tensorflow as tf

args = pickle.load(open("args.pkl", 'rb'))
dst = "vis_image"
image_output = ImageOutput(dst, args)
classes = read_class_names(args.classes)




In [2]:
path = "test.pkl"
with open(path, 'rb') as f:
    data = pickle.load(f)
    

In [3]:
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import tensor_shape
from tensorflow.python.ops import gen_math_ops
from tensorflow.python.ops import gen_array_ops

# @tf.function(jit_compile=True)
def where(condition, x=None, y=None, name=None):
  if x is None and y is None:
    with ops.name_scope(name, "Where", [condition]) as name:
      condition = ops.convert_to_tensor(
          condition, preferred_dtype=tf.dtypes.bool, name="condition")
      return gen_array_ops.where(condition=condition, name=name)
  elif x is not None and y is not None:
    return gen_math_ops.select_v2(condition=condition, t=x, e=y, name=name)
  else:
    raise ValueError("x and y must both be non-None or both be None.")

def boolean_mask(tensor, mask, name="boolean_mask", axis=None):
  def _apply_mask_1d(reshaped_tensor, mask, axis=None):
    """Mask tensor along dimension 0 with a 1-D mask."""
    indices = tf.squeeze(where(mask), axis=[1])
    return tf.gather(reshaped_tensor, indices, axis=axis)

  with ops.name_scope(name, values=[tensor, mask]):
    tensor = ops.convert_to_tensor(tensor, name="tensor")
    mask = ops.convert_to_tensor(mask, name="mask")

    shape_mask = mask.get_shape()
    ndims_mask = shape_mask.ndims
    shape_tensor = tensor.get_shape()
    if ndims_mask == 0:
      raise ValueError("mask cannot be scalar.")
    if ndims_mask is None:
      raise ValueError(
          "Number of mask dimensions must be specified, even if some dimensions"
          " are None.  E.g. shape=[None] is ok, but shape=None is not.")
    axis = 0 if axis is None else axis
    axis_value = tensor_util.constant_value(axis)
    if axis_value is not None:
      axis = axis_value
      shape_tensor[axis:axis + ndims_mask].assert_is_compatible_with(shape_mask)

    leading_size = gen_math_ops.prod(tf.shape(tensor)[axis:axis + ndims_mask], [0])
    tensor = tf.reshape(
        tensor,
        tf.concat([
            tf.shape(tensor)[:axis], [leading_size],
            tf.shape(tensor)[axis + ndims_mask:]
        ], 0))
    # TODO(yongtang): tf.reshape in C++ kernel might have set the shape
    # correctly, so the following may not be needed? It still might be possible
    # that there are some edge case where tensor_util.constant_value resolves
    # more cases than ShapeInference of tf.reshape in C++ kernel.
    if axis_value is not None:
      first_dim = shape_tensor[axis:axis + ndims_mask].num_elements()
      tensor.set_shape(
          tensor_shape.as_shape(shape_tensor[:axis]).concatenate(
              [first_dim]).concatenate(shape_tensor[axis + ndims_mask:]))

    mask = tf.reshape(mask, [-1])
    return _apply_mask_1d(tensor, mask, axis)



In [4]:
# @tf.function(jit_compile=True)
def bmask(tensor, mask):
    for tens in tensor[0]:
        k = lambda : tens
        tf.cond()
    return tensor[0][mask[0]]

def filter(box_xywh, scores, input_shape, score_threshold=0.4):
    scores_max = tf.math.reduce_max(scores, axis=-1)

    mask = scores_max >= score_threshold
    print(mask)
    print(box_xywh)
    
    k = bmask(box_xywh, mask)
    t = boolean_mask(box_xywh, mask)
    print(t==k)

boxes, scores = data["predictions"]
# filter(boxes, scores, tf.constant([416, 416]))

k = tf.constant([1,2])
j = tf.constant([], dtype=tf.dtypes.int32)

tf.concat([k, j], 0)

empty = lambda : tf.constant([], dtype=tf.dtypes.int32)

# f = lambda : tf.constant([1, 2])
# j = tf.less()
# tf.case((j, f))
# tf.gather(k, [0, 1])

In [32]:
@tf.function(jit_compile=True)
def fbox(box_xywh, scores, input_shape, score_threshold=0.4):
    scores_max = tf.math.reduce_max(scores, axis=-1)

    mask = scores_max >= score_threshold
    
    # class_boxes = boolean_mask(box_xywh, mask)
    class_boxes = box_xywh
    # pred_conf = boolean_mask(scores, mask)
    pred_conf = scores 
    class_boxes = tf.reshape(class_boxes, [tf.shape(
        scores)[0], -1, tf.shape(class_boxes)[-1]])
    pred_conf = tf.reshape(pred_conf, [tf.shape(
        scores)[0], -1, tf.shape(pred_conf)[-1]])

    box_xy, box_wh = tf.split(class_boxes, (2, 2), axis=-1)

    input_shape = tf.cast(input_shape, dtype=tf.float32)

    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]

    box_mins = (box_yx - (box_hw / 2.)) / input_shape
    box_maxes = (box_yx + (box_hw / 2.)) / input_shape
    boxes = tf.concat([
        box_mins[..., 0:1],  # y_min
        box_mins[..., 1:2],  # x_min
        box_maxes[..., 0:1],  # y_max
        box_maxes[..., 1:2]  # x_max
    ], axis=-1)
    # boxes = boolean_mask(boxes, mask)
    # pred_conf = boolean_mask(pred_conf, mask)
    # return tf.concat([boxes, pred_conf], axis=-1)
    return (mask, boxes, pred_conf)

def filter_boxes(box_xywh, scores, input_shape, score_threshold=0.4):
    scores_max = tf.math.reduce_max(scores, axis=-1)

    mask = scores_max >= score_threshold
    
    class_boxes = boolean_mask(box_xywh, mask)
    # class_boxes = bmask(box_xywh, mask)
    pred_conf = boolean_mask(scores, mask)
    # pred_conf = scores 
    class_boxes = tf.reshape(class_boxes, [tf.shape(
        scores)[0], -1, tf.shape(class_boxes)[-1]])
    pred_conf = tf.reshape(pred_conf, [tf.shape(
        scores)[0], -1, tf.shape(pred_conf)[-1]])

    box_xy, box_wh = tf.split(class_boxes, (2, 2), axis=-1)

    input_shape = tf.cast(input_shape, dtype=tf.float32)

    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]

    box_mins = (box_yx - (box_hw / 2.)) / input_shape
    box_maxes = (box_yx + (box_hw / 2.)) / input_shape
    boxes = tf.concat([
        box_mins[..., 0:1],  # y_min
        box_mins[..., 1:2],  # x_min
        box_maxes[..., 0:1],  # y_max
        box_maxes[..., 1:2]  # x_max
    ], axis=-1)
    # return tf.concat([boxes, pred_conf], axis=-1)
    return (boxes, pred_conf)

boxes, scores = data["predictions"]

b, s = filter_boxes(boxes, scores, tf.constant([416, 416]))
b = tf.reshape(b, (1, -1, 1, 4))
# print(b, s)


mask, bb, ss = fbox(boxes, scores, tf.constant([416, 416]))
bb = boolean_mask(bb, mask, -1)
bb = tf.reshape(bb, (1, -1, 1, 4))

ss = boolean_mask(ss, mask)
print(b, bb)
print(bb == b[0])

tf.Tensor(
[[[[ 0.33678687  0.0054167   0.7048629   0.17347878]]

  [[ 0.33641636  0.00479237  0.7055864   0.17382424]]

  [[ 0.33436158  0.00442553  0.68800896  0.17412211]]

  [[ 0.00940796 -0.01635606  0.99868894  0.92331284]]

  [[ 0.01132342 -0.02038383  0.9961736   0.9294301 ]]

  [[-0.00652794  0.02315283  0.99992955  0.9572807 ]]

  [[-0.00659334  0.02518533  0.99886024  0.95635736]]

  [[ 0.47455993  0.06099111  0.6905219   0.28493753]]]], shape=(1, 8, 1, 4), dtype=float32) tf.Tensor(
[[[[ 0.33678687  0.0054167   0.7048629   0.1734788 ]]

  [[ 0.33641636  0.00479237  0.7055864   0.17382425]]

  [[ 0.3343616   0.00442553  0.688009    0.17412212]]

  [[ 0.00940796 -0.01635607  0.99868894  0.9233129 ]]

  [[ 0.01132342 -0.02038383  0.9961737   0.9294302 ]]

  [[-0.00652794  0.02315283  0.9999296   0.9572807 ]]

  [[-0.00659334  0.02518533  0.99886024  0.9563574 ]]

  [[ 0.47455996  0.06099112  0.69052196  0.28493753]]]], shape=(1, 8, 1, 4), dtype=float32)
tf.Tensor(
[[[[ True  Tr

In [17]:
from tfpipe.core.utils import draw_bbox

boxes, scores = data["predictions"]

boxes, scores = filter_boxes(boxes, scores, tf.constant([416, 416]))
# boxes=boxes[0]
print(boxes)

boxes = tf.reshape(boxes, (1, -1, 1, 4))
print(boxes)
scores = tf.reshape(scores, (1, -1, tf.shape(scores)[-1]))

boxes, scores, classes, valid_detections = tf.image.combined_non_max_suppression(
    boxes,
    scores,
    max_output_size_per_class=50,
    max_total_size=50,
    iou_threshold=args.iou,
    score_threshold=args.score
)

pred_bbox = [boxes.numpy(), scores.numpy(), classes.numpy(),
                valid_detections.numpy()]


annotated_image = draw_bbox(
    data["image"].copy(), pred_bbox, classes)

data[dst] = annotated_image

image_output.map(data)


tf.Tensor(
[[[ 0.33678687  0.0054167   0.7048629   0.17347878]
  [ 0.33641636  0.00479237  0.7055864   0.17382424]
  [ 0.33436158  0.00442553  0.68800896  0.17412211]
  [ 0.00940796 -0.01635606  0.99868894  0.92331284]
  [ 0.01132342 -0.02038383  0.9961736   0.9294301 ]
  [-0.00652794  0.02315283  0.99992955  0.9572807 ]
  [-0.00659334  0.02518533  0.99886024  0.95635736]
  [ 0.47455993  0.06099111  0.6905219   0.28493753]]], shape=(1, 8, 4), dtype=float32)
tf.Tensor(
[[[[ 0.33678687  0.0054167   0.7048629   0.17347878]]

  [[ 0.33641636  0.00479237  0.7055864   0.17382424]]

  [[ 0.33436158  0.00442553  0.68800896  0.17412211]]

  [[ 0.00940796 -0.01635606  0.99868894  0.92331284]]

  [[ 0.01132342 -0.02038383  0.9961736   0.9294301 ]]

  [[-0.00652794  0.02315283  0.99992955  0.9572807 ]]

  [[-0.00659334  0.02518533  0.99886024  0.95635736]]

  [[ 0.47455993  0.06099111  0.6905219   0.28493753]]]], shape=(1, 8, 1, 4), dtype=float32)


{'image_id': '.\\data\\images\\girl.png',
 'image': array([[[206, 153, 112],
         [206, 153, 112],
         [206, 153, 112],
         ...,
         [164, 111,  72],
         [164, 111,  72],
         [164, 111,  72]],
 
        [[206, 153, 112],
         [206, 153, 112],
         [206, 153, 112],
         ...,
         [164, 111,  72],
         [164, 111,  72],
         [164, 111,  72]],
 
        [[204, 152, 111],
         [206, 153, 112],
         [206, 153, 112],
         ...,
         [164, 111,  72],
         [164, 111,  72],
         [164, 111,  72]],
 
        ...,
 
        [[113,  49,  23],
         [112,  47,  23],
         [112,  47,  23],
         ...,
         [127,  87,  56],
         [127,  87,  56],
         [127,  87,  56]],
 
        [[110,  46,  22],
         [110,  46,  22],
         [110,  46,  22],
         ...,
         [126,  85,  56],
         [126,  85,  56],
         [126,  85,  56]],
 
        [[109,  46,  23],
         [109,  46,  22],
         [108,  4